In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
import os
import requests
import pandas

from rdkit import Chem
from rdkit.Chem.Draw import IPythonConsole, rdMolDraw2D
from IPython.display import HTML

no_wrap_div = '<div style="white-space: nowrap">{}{}</div>'

#SERVICE_BASE_URL = 'http://mdstudio-smartcyp.dev.openrisknet.org/'
SERVICE_BASE_URL = 'http://localhost:8081'

In [ ]:
mol_files = ['/Volumes/Arabica/WorkTeaching/bproject_X_430533_2019/ligands_spores/aminopyrine.mol2',
             '/Volumes/Arabica/WorkTeaching/bproject_X_430533_2019/ligands_spores/estradiol.mol2',
             '/Volumes/Arabica/WorkTeaching/bproject_X_430533_2019/ligands_spores/methadone.mol2']
# 12, 13 2D6
# 8, 9, 11, 15 2D6
# 20, 21 2D6

In [ ]:
def show_predictions(mol, prob, cutoff=0.75):
    """
    Build 2D depiction of rdkit molecule
    Label atoms with index number and SOM prediction probabilities if
    above cutoff.
    """
    
    prob_dict = dict([(int(a.split('.')[1]), b) for a,b in prob.items()])
    
    Chem.rdDepictor.Compute2DCoords(mol)
    for atom in mol.GetAtoms():
        idx = atom.GetIdx() + 1
        atom.SetProp('molAtomMapNumber', '{0}-{1:.2f}'.format(idx, prob_dict.get(idx, 0.0)))
    
    drawer = rdMolDraw2D.MolDraw2DSVG(400,250)
    drawer.DrawMolecule(mol, highlightAtoms=[i-1 for i in prob_dict if prob_dict[i] >= cutoff])
    drawer.FinishDrawing()
    return drawer.GetDrawingText().replace('svg:','')

In [ ]:
data = {'cyp': '2D6', 'cluster_structures': 50}
for mol in mol_files:
    files = {'ligand_file': open(mol, 'rb').read()}
    
    # Get SMARTCyp prediction
    response = requests.post('{0}/som_prediction'.format(SERVICE_BASE_URL), files=files, data=data)
    response_json = response.json() 
    response_df = pandas.DataFrame.from_dict(response_json, orient='index')
    response_df_prob = response_df[['Docking', 'SMARTCyp']].fillna(0).round(3)
    
    # Draw molecule
    rdmol = Chem.rdmolfiles.MolFromMol2File(mol)
    dock_svg = show_predictions(rdmol, response_df_prob['Docking'])
    smart_svg = show_predictions(rdmol, response_df_prob['SMARTCyp'])
    
    display(HTML('<h3 align="center">{0}</h3>'.format(os.path.basename(mol).split('.')[0])))
    display(HTML(no_wrap_div.format(dock_svg, smart_svg)))
    display(response_df_prob.T)

In [ ]:
p = "http://fame3.dev.openrisknet.org/predict"
response = requests.get(p, params={'smiles': 'C1=CC(=C(C(=C1)C(=O)C2=CC=C(C=C2)Br)N)CC(=O)O'},
                        headers={"accept" : "application/json"})
response_json = response.json() 
print(response_json)